# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** First steps with data

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import tensorflow as tf
import sys
from keras.utils import to_categorical
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold, KFold

2023-06-13 15:32:54.964375: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Import models
import cnn_model

In [3]:
# Initializing test set
np.random.seed(3)
data_org = pd.read_csv('final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)

data_org

In [4]:
sum_vec = data_org.iloc[:, :9].sum(axis=1)
data2 = data_org.iloc[:, :9].divide(sum_vec, axis='index')
data = pd.concat([data2, data_org['seq']], axis=1)
data

In [5]:
# One hot encode the 'seq' attribute of the above table
mapping = { 
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3
            }
one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one_hot_karas = data['seq'].apply(one_hot_encode_lam)

data_one_hot_karas

# Now just injecting this modified 'seq' back into the pandas frame 
data_new_karas =  pd.concat([data.iloc[:, :9], data_one_hot_karas], axis=1)

data_new_karas

In [6]:
# Splitting for 5fold

folds_total = 5

# Only Truncation?
X = data_new_karas['seq'].apply(lambda x: x[:64])
# X = data_new_karas['seq']

# y = pd.DataFrame(columns=['Labels'])

# X = tf.data.Dataset.from_tensor_slices(dict(X))

y = data_new_karas.iloc[:, :9].values.tolist()
# y = y.apply(lambda x: np.array(x))

# y = tf.data.Dataset.from_tensor_slices(dict(y))

X = np.stack(X).transpose((0, 2, 1))
y = np.stack(y)

# X = tf.convert_to_tensor(X)
# y = tf.convert_to_tensor(y)

# kf = KFold(n_splits=folds_total, shuffle=True, random_state=1234)

# folds = kf.split(X, y)

kf = KFold(n_splits=folds_total, shuffle=True, random_state=1)

folds = kf.split(X,y)

print(X.shape)
print(y.shape)
# folds now contains a list of lists. Each sublist contains all the indices for the pandas data entries to be used in the respective fold

(13810, 4, 64)
(13810, 9)


In [ ]:
# TODO: what is this?

template = [0] * 24  # dim([a,c,g,t]) * dim([f,t,i,h,m,s])
combined_encoding = OrderedDict()
combined_encoding['UNK'] = template
for i, (key_seq, key_ann) in enumerate(
        itertools.product(['A', 'C', 'G', 'T', 'N'], ['F', 'T', 'I', 'H', 'M', 'S'])):
    tmp = template.copy()
    if key_seq == 'N':
        for n in ['A', 'C', 'G', 'T']:
            tmp[np.nonzero(combined_encoding[n + key_ann])[0][0]] = 0.25
        combined_encoding[key_seq + key_ann] = tmp
    else:
        tmp[i] = 1  # normal one-hot encoding as it is...
        combined_encoding[key_seq + key_ann] = tmp
encoding_keys = list(combined_encoding.keys())
encoding_vectors = np.array(list(combined_encoding.values()))
encoding_vectors

### **2** |  Importing models...

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [7]:
# Import NN
from RNAtracker import RNATracker

# TODO: do a normal CNN


# Set paths for model output
try:
    os.mkdir('model_outputs/')
except Exception as e:
    print("Output folder already exists")

model_output_folder = 'model_outputs/'



Output folder already exists


In [8]:

# TODO: with understand what we have to predict, we can allocate X and y
# Also: the kwargsvalues are hyperparameters of which we will select default values from the RNAtracker repo

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

print("Start training...")

for i, (train_indices, test_indices) in enumerate(folds):
    print('Evaluating KFolds {}/{}'.format(str(i + 1), str(folds_total)))
    # model = RNATracker(max_len, nb_classes, model_output_folder, kfold_index=i)
    #model.build_model(nb_filters=kwargs['nb_filters'], filters_length=kwargs['filters_length'],
    #                          pooling_size=kwargs['pooling_size'], lstm_units=kwargs['lstm_units'],
    #                          embedding_vec=encoding_vectors)

    # model.build_model_advanced_masking(nb_filters=kwargs['nb_filters'],
     #                                              filters_length=kwargs['filters_length'],
      #                                             pooling_size=kwargs['pooling_size'],
       #                                            lstm_units=kwargs['lstm_units'],
        #                                           embedding_vec=encoding_vectors)
    

    # model.train(X[train_indices], y[train_indices], batch_size, kwargs['epochs'])

    # model.evaluate(X[test_indices], y[test_indices], dataset)

    # K.clear_session()
    model = cnn_model.CNN_MLRG().build_model()

    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_output_folder + f"cnn{i}.h5",
							monitor='val_accuracy', verbose=1,
							save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    history = model.fit(X[train_indices], y[train_indices], epochs=3,
                        callbacks=callbacks_list, validation_data=(X[test_indices], y[test_indices]))

    model.load_weights(f"/model_outputs/cnn{i}.h5")

    results = model.evaluate(X[test_indices])
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

    tf.keras.backend.clear_session()

print("Training done!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 32)             6176      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 32)            0         
 )                                                               
                                                                 
 dense (Dense)               (None, 1, 9)              297       
                                                                 
 flatten (Flatten)           (None, 9)                 0         
                                                                 
Total params: 6,473
Trainable params: 6,473
Non-trainable params: 0
_________________________________________________________________
